In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import scipy
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import graph
import threading
from graph import get_renormalized_adj_matrix, get_chebyshev_polynomials

from scipy import sparse
from models.indrnn_model import indrnn_model
from models.gcgru_model import gcgru_model
from models.lstm_model import lstm_model
from models.var_model import var_model
%load_ext autoreload
%autoreload 1

### Graph construction

In [2]:
config = dict(
    model_name = 'lstm2',
    batch_size = 32,
    n_input_steps = 40,
    n_output_steps = 12,
    n_steps = 15000,
    eval_secs = 120)

params = dict(
    n_layers = 2,
    n_nodes = 256,
    hidden_size = 64,
    n_input_steps = 40,
    n_output_steps = 12,
    learning_rate = 1e-3,
    adj = 'data/processed/adj_256.npy',
    sampling_start = 5000
)

In [3]:
def lr_decay_fn(lr,step):
    step = tf.cast(step, tf.float32)
    step = tf.maximum(step - 4000,0.)
    ticks = step // 1000
    return lr / (np.sqrt(10)**ticks)

def model_fn(features, labels, mode, params, config):
    adj = np.load(params.adj)
    
    conv_matrix = tf.constant(get_renormalized_adj_matrix(adj-np.eye(params.n_nodes)), dtype = tf.float32)
    
    conv_matrix2 = tf.constant(get_chebyshev_polynomials(adj, 2), dtype = tf.float32)
    
#     predictions = indrnn_model(features, labels, conv_matrix, mode, params)
    predictions = gcgru_model(features, labels, conv_matrix2, mode, params)
#     predictions = lstm_model(features, labels, mode, params)
#     predictions = var_model(features, labels, mode, params)
    if mode == tf.estimator.ModeKeys.TRAIN:
        loss = tf.losses.mean_squared_error(labels = labels, predictions= predictions)
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.train.get_or_create_global_step(),
            learning_rate=params.learning_rate,
            optimizer= tf.train.AdamOptimizer,
            learning_rate_decay_fn = lr_decay_fn,
            clip_gradients=0.25)
        specs = dict(
            mode=mode,
            loss=loss,
            train_op=train_op,
        )
    if mode == tf.estimator.ModeKeys.EVAL:
        for x in tf.trainable_variables():
            tf.summary.histogram(x.name.replace(':','_'), x)
        mse = tf.metrics.mean_squared_error(labels = labels, predictions= predictions)
#         rmse = tf.metrics.root_mean_squared_error(labels, predictions)
#         mae = tf.metrics.mean_absolute_error(labels, predictions)    
        
        metrics = {'metrics/MSE' : mse}#, 'metrics/MAE' : mae, 'metrics/RMSE' : rmse}
        loss = tf.losses.mean_squared_error(labels = labels, predictions= predictions)
        hook = tf.train.SummarySaverHook(save_steps = 1,output_dir=run_config.model_dir, summary_op=tf.summary.merge_all())
        specs = dict(
            mode=mode,
            loss=loss,
            eval_metric_ops=metrics,
            evaluation_hooks = [hook]
        )
    if mode == tf.estimator.ModeKeys.PREDICT:
        specs = dict(
            mode=mode,
            predictions=predictions
        )
    print(np.sum([np.prod(v.get_shape().as_list()) for v in tf.trainable_variables()]))

    return tf.estimator.EstimatorSpec(**specs)

In [4]:
def input_fn(mode,
             dataset,
             batch_size,
             input_steps,
             output_steps,
             sample_buffer_size=1000):
    num_epochs = None if mode == tf.estimator.ModeKeys.TRAIN else 1
    dataset = tf.data.Dataset.from_tensor_slices(dataset.astype(np.float32))
    if mode != tf.estimator.ModeKeys.PREDICT:
        dataset = dataset.apply(tf.contrib.data.sliding_window_batch(input_steps + output_steps))
        dataset = dataset.map(lambda x: (x[:input_steps], x[input_steps:]))
        dataset = dataset.shuffle(sample_buffer_size)
    else:
        dataset = dataset.apply(tf.contrib.data.sliding_window_batch(input_steps)) 
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    return dataset

In [5]:
hparams = tf.contrib.training.HParams(**params)
model_dir = os.path.join('output', config['model_name'])

run_config = tf.estimator.RunConfig(model_dir=model_dir, save_summary_steps=10, save_checkpoints_steps=1000, 
                                       save_checkpoints_secs = None)
estimator = tf.estimator.Estimator(model_fn,config = run_config, params = hparams)

INFO:tensorflow:Using config: {'_model_dir': 'output\\lstm2', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000298C9C26978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
X_train = np.load('data/processed/train_256.npy')
X_valid = np.load('data/processed/valid_256.npy')

train_input_fn = lambda : input_fn(tf.estimator.ModeKeys.TRAIN, X_train, config['batch_size'], 
                                   config['n_input_steps'], config['n_output_steps'], 1000)
val_input_fn = lambda : input_fn(tf.estimator.ModeKeys.EVAL, X_valid, config['batch_size']*4, 
                                 config['n_input_steps'], config['n_output_steps'], 1000)
train_spec = tf.estimator.TrainSpec(train_input_fn, max_steps = config['n_steps'])
eval_spec = tf.estimator.EvalSpec(val_input_fn, steps = None, throttle_secs = config['eval_secs'])

In [7]:
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 120 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
223168
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output\lstm2\model.ckpt-2065


NotFoundError: Key OptimizeLoss/decoder/output_projection_wrapper/multi_rnn_cell/cell_0/gcgru/biases1/Adam not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]
	 [[Node: save/RestoreV2/_79 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_84_save/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save/RestoreV2', defined at:
  File "c:\users\nprok\anaconda3\envs\tf-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\nprok\anaconda3\envs\tf-gpu\lib\runpy.py", line 85, in _run_code
   

## Evaluation

In [ ]:
import pickle
with open('data/processed/std_scaler_256.pkl', 'rb') as file:
    std_scaler = pickle.load(file)
X_test = np.load('data/processed/test_256.npy')
predict_input_fn = lambda : input_fn(tf.estimator.ModeKeys.PREDICT, X_test[:-12], 256, 40, 12, 1000)

In [ ]:
# ckpt = 'output\lstm\model.ckpt-10081'

In [ ]:
y_true = [[],[],[],[]]
y_pred = [[],[],[],[]]
for x in estimator.predict(predict_input_fn):#, checkpoint_path=ckpt):
    y_pred[0].append(std_scaler.inverse_transform(x[0]))
    y_pred[1].append(std_scaler.inverse_transform(x[2]))
    y_pred[2].append(std_scaler.inverse_transform(x[5]))
    y_pred[3].append(std_scaler.inverse_transform(x[11]))

#[dd:-dd] - offsets containing ground truth values in dataset, e.g. 15 minute horizont has offsets [42:-9],
#we use first 40 steps as input, 42 step is a prediction. In seq2seq we pick 40 inputs and predict 12 outputs,
#thats why we can't predict values past :-9
y_true[0] = [std_scaler.inverse_transform(x.flatten()) for x in X_test[40:-11]]
y_true[1] = [std_scaler.inverse_transform(x.flatten()) for x in X_test[42:-9]]
y_true[2] = [std_scaler.inverse_transform(x.flatten()) for x in X_test[45:-6]]
y_true[3] = [std_scaler.inverse_transform(x.flatten()) for x in X_test[51:]]

y_true = np.array(y_true)
y_pred = np.array(y_pred)

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
"LSTM"
print('MSE for 5 15 30 60 mins')
for i in range(4):
    print(np.sqrt(sklearn.metrics.mean_squared_error(y_true[i].reshape(-1,1), y_pred[i].reshape(-1,1))))
#0.8 10 29

In [ ]:
print('MAE for 5 15 30 60 mins')
for i in range(4):
    print(sklearn.metrics.mean_absolute_error(y_true[i].reshape(-1,1), y_pred[i].reshape(-1,1)))

In [ ]:
print('MAPE for 5 15 30 60 mins')
for i in range(4):
    print(mean_absolute_percentage_error(y_true[i].reshape(-1,1), y_pred[i].reshape(-1,1)))

In [ ]:
import datetime
import matplotlib.dates as mdates

def plot(i):
    #plt.figure(figsize=(11,7))
    x = [datetime.datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) + datetime.timedelta(minutes=i*5) for i in range(280)]
    plt.plot(x,y_true[2][288*i:288*(i+1)][:,2][:280] ,label = 'Данные с датчиков', linewidth=2)
    plt.plot(x,y_pred[2][288*i:288*(i+1)][:,2][:280] ,label = 'Прогноз', linewidth=2.0)
    plt.ylabel('Скорость, м/ч')
    plt.legend(loc='center left')
    plt.gcf().autofmt_xdate()
    myFmt = mdates.DateFormatter('%H:%M')
    plt.gca().xaxis.set_major_formatter(myFmt)
   # plt.savefig('fig2.png', dpi = 300)

In [ ]:
y_true.shape

In [ ]:
plot(36)

In [ ]:
#output\ind_rnn_5layers_bs32_tanh_hidden64_k5000_lr10000\eval 5.05
#output\ind_rnn_5layers_bs32_relu_hidden64_k5000_lr10000_2proj\eval 5.09
#output\ind_rnn_5layers_bs32_relu_hidden64_k5000_lr10000_stat 5.14